In [1]:
import gymnasium as gym
import numpy as np
import time
from stable_baselines3 import A2C
from stable_baselines3.common.torch_layers import BaseFeaturesExtractor
from stable_baselines3.common.env_util import make_vec_env
from stable_baselines3.common.vec_env import VecFrameStack, VecTransposeImage, DummyVecEnv
import torch.nn as nn

In [9]:
class CustomObservationWrapper(gym.ObservationWrapper):
    def __init__(self, env):
        super(CustomObservationWrapper, self).__init__(env)
        # Define the new observation space here, e.g., an image
        self.observation_space = gym.spaces.Box(low=0, high=255, shape=(400, 600, 3), dtype=np.uint8)

    def observation(self, observation):
        # Process the observation and return it
        # For example, capturing an image from the environment
        return self.env.render()

# Create and wrap the environment
env = gym.make('CartPole-v1', render_mode="rgb_array")
env = CustomObservationWrapper(env)

# Wrap it into a DummyVecEnv for compatibility with SB3
env = DummyVecEnv([lambda: env])
env = VecTransposeImage(env)  # Required for CNN policies in SB3

model = A2C("CnnPolicy", env, verbose=1)
model.learn(total_timesteps=10000, progress_bar=True)


2024-01-25 11:48:11.074892: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-01-25 11:48:11.074879: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-01-25 11:48:11.074885: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-01-25 11:48:11.074907: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-01-25 11:48:11.074909: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory

Using cpu device
------------------------------------
| rollout/              |          |
|    ep_len_mean        | 27       |
|    ep_rew_mean        | 27       |
| time/                 |          |
|    fps                | 2950     |
|    iterations         | 100      |
|    time_elapsed       | 1        |
|    total_timesteps    | 4000     |
| train/                |          |
|    entropy_loss       | -0.645   |
|    explained_variance | 0.218    |
|    learning_rate      | 0.0007   |
|    n_updates          | 99       |
|    policy_loss        | 1.02     |
|    value_loss         | 15.7     |
------------------------------------
------------------------------------
| rollout/              |          |
|    ep_len_mean        | 49.6     |
|    ep_rew_mean        | 49.6     |
| time/                 |          |
|    fps                | 3008     |
|    iterations         | 200      |
|    time_elapsed       | 2        |
|    total_timesteps    | 8000     |
| train/             

In [8]:
episodes = 10

env = gym.make('CartPole-v1', render_mode="rgb_array")
env = CustomObservationWrapper(env)

# Wrap it into a DummyVecEnv for compatibility with SB3
env = DummyVecEnv([lambda: env])
env = VecTransposeImage(env) 

for episode in range(1, episodes+1):    
    observation = env.reset()
    finished = False
    accumulated_reward = 0
    
    while not finished:
        action = model.predict(observation)[0]
        observation, reward, terminated, truncated = env.step(action)
        accumulated_reward += reward

        terminated_flag = terminated[0]
        truncated_flag = truncated[0].get('TimeLimit.truncated', False)
        
        finished = terminated_flag or truncated_flag

        if finished:
            env.reset()
            print("Episode {} terminated".format(episode))
            print("Accumulated Reward: {}".format(accumulated_reward))

env.close()

Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
Episode 1 terminated
Accumulated Reward: 500.0
Episode 2 terminated
Accumulated Reward: 500.0
Episode 3 terminated
Accumulated Reward: 500.0
Episode 4 terminated
Accumulated Reward: 500.0
Episode 5 terminated
Accumulated Reward: 500.0


KeyboardInterrupt: 